In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import scipy.stats as stats
import warnings 
warnings.filterwarnings("ignore")

import math

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('display.max_columns',100)

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Train Data

In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
train['MSSubClass'] = train['MSSubClass'].astype(object)

In [4]:
X_train=train.drop(['Id', 'SalePrice'], axis=1).copy()
y_train=train['SalePrice'].copy()

## Missing Value Handling

In [5]:
def missing_info(data):
    
    missing={'Feature':data.columns,
             'Total_NA':data.isnull().sum(),
             'Proportion_NA':data.isnull().sum() / len(data)}
    missing=pd.DataFrame(missing)
    missing = missing.sort_values('Total_NA', ascending=False)
    return missing[missing['Total_NA']>0].style.hide_index()

In [6]:
missing_info(X_train)

Feature,Total_NA,Proportion_NA
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageQual,81,0.055479
GarageCond,81,0.055479


In [7]:
#missing value num data
missing_info(X_train.select_dtypes(include=[int,float]))

Feature,Total_NA,Proportion_NA
LotFrontage,259,0.177397
GarageYrBlt,81,0.055479
MasVnrArea,8,0.005479


In [8]:
#num data area feature
X_train['LotFrontage'] = X_train['LotFrontage'].fillna(0)
X_train['MasVnrArea'] = X_train['MasVnrArea'].fillna(0)

In [9]:
#num data year that not has garage feature
X_train['GarageYrBlt']= X_train['GarageYrBlt'].fillna(-9999)

In [10]:
#missing value cat data
missing_info(X_train.select_dtypes(exclude=[int,float]))

Feature,Total_NA,Proportion_NA
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
GarageType,81,0.055479
GarageFinish,81,0.055479
GarageCond,81,0.055479
GarageQual,81,0.055479
BsmtFinType2,38,0.026027


In [11]:
#cat feature with NA represent not has feature
NA_no_feature=['Alley','MasVnrType','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2', 
               'FireplaceQu','GarageType','GarageFinish', 'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [12]:
#fillna cat feature with NA represent not has feature
for col in NA_no_feature:
    X_train[col]=X_train[col].fillna('NA')
#fill na Electrical with train mode in EDA
X_train['Electrical']=X_train['Electrical'].fillna('SBrkr')

In [13]:
#missing value cat data
missing_info(X_train)

Feature,Total_NA,Proportion_NA


## Additional Feature

In [14]:
#Total sqaure feet house floor
X_train['TotalFlArea'] = X_train['1stFlrSF']+X_train['2ndFlrSF']
#ratio 2sn floor with 1stfloor
X_train['2ndFlRatio'] = X_train['2ndFlrSF']/X_train['1stFlrSF']
#ratio living area
X_train["LivLotRatio"] = X_train['GrLivArea'] / X_train['LotArea']
#mean overall qual and cond house
X_train['Home_QualCond'] = (X_train['OverallQual'] + X_train['OverallCond'])/2
#Avarege room  area
X_train['MeanRoomArea'] =X_train['TotalFlArea']/X_train['TotRmsAbvGrd']
#House Age when sold
X_train['AgeSold']=X_train['YrSold']-X_train['YearBuilt']
#get remodeled or Additinal
X_train['GetRemodelAdd']=X_train['YearRemodAdd']-X_train['YearBuilt']
#season get sold
season_ranges = [0, 2, 5, 8, 11, 12]
season_labels = ['Winter', 'Spring', 'Summer', 'Autumn', 'Winter']
X_train['SeasonSold'] = pd.cut(X_train['MoSold'], bins=season_ranges, labels=season_labels, right=False, ordered=False)

## Categorical Encoding

### Ordinal Encoding

In [15]:
#Qual and Cond value
Qual_Cond_Values={'Ex':5, 'Gd':4, 'TA':3,'Fa':2, 'Po':1,'NA':0}
Qual_Cond_Features = ['ExterQual','ExterCond','BsmtQual','BsmtCond','FireplaceQu','GarageQual','GarageCond','PoolQC', 'KitchenQual']
for col in Qual_Cond_Features:
    X_train[col]=X_train[col].replace(Qual_Cond_Values)

In [16]:
#BsmtExposure values
BsmtExposure_values={'Gd':3, 'Av':2,'Mn':1, 'No':0,'NA':-1}
X_train['BsmtExposure']=X_train['BsmtExposure'].replace(BsmtExposure_values)

In [17]:
#CentralAir condition values
CentralAir_values={'Y':1,'N':0}
X_train['CentralAir']=X_train['CentralAir'].replace(CentralAir_values)

### OneHot Encoding

In [18]:
X_train = pd.get_dummies(X_train)

In [19]:
X_train.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,TotalFlArea,2ndFlRatio,LivLotRatio,Home_QualCond,...,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SeasonSold_Autumn,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter
0,65.0,8450,7,5,2003,2003,196.0,4,3,4,3,0,706,0,150,856,1,856,854,0,1710,1,0,2,1,3,1,4,8,0,0,2003.0,2,548,3,3,0,61,0,0,0,0,0,0,2,2008,1710,0.997664,0.202367,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
1,80.0,9600,6,8,1976,1976,0.0,3,3,4,3,3,978,0,284,1262,1,1262,0,0,1262,0,1,2,0,3,1,3,6,1,3,1976.0,2,460,3,3,298,0,0,0,0,0,0,0,5,2007,1262,0.000000,0.131458,7.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
2,68.0,11250,7,5,2001,2002,162.0,4,3,4,3,1,486,0,434,920,1,920,866,0,1786,1,0,2,1,3,1,4,6,1,3,2001.0,2,608,3,3,0,42,0,0,0,0,0,0,9,2008,1786,0.941304,0.158756,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
3,60.0,9550,7,5,1915,1970,0.0,3,3,3,4,0,216,0,540,756,1,961,756,0,1717,1,0,1,0,3,1,4,7,1,4,1998.0,3,642,3,3,0,35,272,0,0,0,0,0,2,2006,1717,0.786681,0.179791,6.0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0
4,84.0,14260,8,5,2000,2000,350.0,4,3,4,3,2,655,0,490,1145,1,1145,1053,0,2198,1,0,2,1,4,1,4,9,1,3,2000.0,3,836,3,3,192,84,0,0,0,0,0,0,12,2008,2198,0.919651,0.154137,6.5,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [20]:
X_train.shape

(1460, 287)

# Test Data

In [21]:
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [22]:
print(set(test['MSSubClass'])-set(train['MSSubClass']))

{150}


In [23]:
print(set(train['MSSubClass'])-set(test['MSSubClass']))

set()


In [24]:
test['MSSubClass'] = test['MSSubClass'].replace({150:50})

In [25]:
X_test = test.drop('Id', axis=1).copy()

In [26]:
X_test['MSSubClass'] = X_test['MSSubClass'].astype(str).astype(object)

## Missing Value Handling

In [27]:
missing_info(X_test)

Feature,Total_NA,Proportion_NA
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461
GarageQual,78,0.053461
GarageCond,78,0.053461


In [28]:
#same method with train data handling
#num data area feature
X_test['LotFrontage'] = X_test['LotFrontage'].fillna(0)
X_test['MasVnrArea'] = X_test['MasVnrArea'].fillna(0)

#num data year that not has garage feature
X_test['GarageYrBlt']= X_test['GarageYrBlt'].fillna(-9999)

#fillna cat feature with NA represent not has feature
for col in NA_no_feature:
    X_test[col]=X_test[col].fillna('NA')

In [29]:
missing_info(X_test)

Feature,Total_NA,Proportion_NA
MSZoning,4,0.002742
BsmtFullBath,2,0.001371
BsmtHalfBath,2,0.001371
Utilities,2,0.001371
Functional,2,0.001371
BsmtFinSF1,1,0.000685
KitchenQual,1,0.000685
Exterior1st,1,0.000685
GarageCars,1,0.000685
GarageArea,1,0.000685


In [30]:
missing_info(X_test.select_dtypes(include=[int,float]))

Feature,Total_NA,Proportion_NA
BsmtHalfBath,2,0.001371
BsmtFullBath,2,0.001371
TotalBsmtSF,1,0.000685
GarageCars,1,0.000685
GarageArea,1,0.000685
BsmtFinSF1,1,0.000685
BsmtFinSF2,1,0.000685
BsmtUnfSF,1,0.000685


### missing value is in garage and or basement feature

In [31]:
#for missing value in garage
X_test[X_test['GarageCars'].isna()][['GarageQual','GarageCars', 'GarageArea','BsmtQual','BsmtHalfBath','BsmtFullBath','BsmtUnfSF','BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF']]

,GarageQual,GarageCars,GarageArea,BsmtQual,BsmtHalfBath,BsmtFullBath,BsmtUnfSF,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF
1116,NA,NaN,NaN,Gd,0.0,0.0,311.0,548.0,0.0,859.0


In [32]:
#NA in GarageQual = no garage
#fill missing value in garage feature
X_test['GarageCars']=X_test['GarageCars'].fillna(0)
X_test['GarageArea']=X_test['GarageArea'].fillna(0)

In [33]:
#for missing value in basement
X_test[X_test['BsmtHalfBath'].isna()][['BsmtQual','BsmtHalfBath','BsmtFullBath','BsmtUnfSF','BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF']]

,BsmtQual,BsmtHalfBath,BsmtFullBath,BsmtUnfSF,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF
660,NA,NaN,NaN,NaN,NaN,NaN,NaN
728,NA,NaN,NaN,0.0,0.0,0.0,0.0


In [34]:
#NA in GarageQual = no basement
#fill missing value in besement feature
X_test['BsmtHalfBath']=X_test['BsmtHalfBath'].fillna(0)
X_test['BsmtFullBath']=X_test['BsmtFullBath'].fillna(0)
X_test['BsmtUnfSF']=X_test['BsmtUnfSF'].fillna(0)
X_test['BsmtFinSF1']=X_test['BsmtFinSF1'].fillna(0)
X_test['BsmtFinSF2']=X_test['BsmtFinSF2'].fillna(0)
X_test['TotalBsmtSF']=X_test['TotalBsmtSF'].fillna(0)

In [35]:
missing_info(X_test.select_dtypes(include=[int,float]))

Feature,Total_NA,Proportion_NA


In [36]:
missing_info(X_test.select_dtypes(exclude=[int,float]))

Feature,Total_NA,Proportion_NA
MSZoning,4,0.002742
Functional,2,0.001371
Utilities,2,0.001371
SaleType,1,0.000685
Exterior2nd,1,0.000685
Exterior1st,1,0.000685
KitchenQual,1,0.000685


In [37]:
for col in X_test.select_dtypes(exclude=[int,float]):
       X_test[col] = X_test[col].fillna(X_test[col].mode()[0])

In [38]:
missing_info(X_test.select_dtypes(exclude=[int,float]))

Feature,Total_NA,Proportion_NA


## Additional Feature

In [39]:
#Total sqaure feet house floor
X_test['TotalFlArea'] = X_test['1stFlrSF']+X_test['2ndFlrSF']
#ratio 2sn floor with 1stfloor
X_test['2ndFlRatio'] = X_test['2ndFlrSF']/X_test['1stFlrSF']
#ratio living area
X_test["LivLotRatio"] = X_test['GrLivArea'] / X_test['LotArea']
#mean overall qual and cond house
X_test['Home_QualCond'] = (X_test['OverallQual'] + X_test['OverallCond'])/2
#Avarege room  area
X_test['MeanRoomArea'] =X_test['TotalFlArea']/X_test['TotRmsAbvGrd']
#House Age when sold
X_test['AgeSold']=X_test['YrSold']-X_test['YearBuilt']
#get remodeled or Additinal
X_test['GetRemodelAdd']=X_test['YearRemodAdd']-X_test['YearBuilt']
#season get sold
season_ranges = [0, 2, 5, 8, 11, 12]
season_labels = ['Winter', 'Spring', 'Summer', 'Autumn', 'Winter']
X_test['SeasonSold'] = pd.cut(X_test['MoSold'], bins=season_ranges, labels=season_labels, right=False, ordered=False)

## Categorical Encoding

### Ordinal Encoding

In [40]:
#Qual and Cond value
Qual_Cond_Values={'Ex':5, 'Gd':4, 'TA':3,'Fa':2, 'Po':1,'NA':0}
Qual_Cond_Features = ['ExterQual','ExterCond','BsmtQual','BsmtCond','FireplaceQu','GarageQual','GarageCond','PoolQC', 'KitchenQual']
for col in Qual_Cond_Features:
    X_test[col]=X_test[col].replace(Qual_Cond_Values)
    
#BsmtExposure values
BsmtExposure_values={'Gd':3, 'Av':2,'Mn':1, 'No':0,'NA':-1}
X_test['BsmtExposure']=X_test['BsmtExposure'].replace(BsmtExposure_values)

#CentralAir condition values
CentralAir_values={'Y':1,'N':0}
X_test['CentralAir']=X_test['CentralAir'].replace(CentralAir_values)

### OneHot Encoding

In [41]:
X_test = pd.get_dummies(X_test)

# Feature Columns Checking

In [42]:
print('X_train shape', X_train.shape)
print('X_test shape', X_test.shape)

X_train shape (1460, 287)
X_test shape (1459, 271)


In [43]:
set(X_train.columns)-set(X_test.columns)

{'Condition2_RRAe',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'Electrical_Mix',
 'Exterior1st_ImStucc',
 'Exterior1st_Stone',
 'Exterior2nd_Other',
 'Heating_Floor',
 'Heating_OthW',
 'HouseStyle_2.5Fin',
 'MiscFeature_TenC',
 'RoofMatl_ClyTile',
 'RoofMatl_Membran',
 'RoofMatl_Metal',
 'RoofMatl_Roll',
 'Utilities_NoSeWa'}

In [44]:
set(X_test.columns)-set(X_train.columns)

set()

In [45]:
missing_columns=['Condition2_RRAe','Condition2_RRAn','Condition2_RRNn','Electrical_Mix','Exterior1st_ImStucc','Exterior1st_Stone',
                 'Exterior2nd_Other','Heating_Floor','Heating_OthW','HouseStyle_2.5Fin','MiscFeature_TenC','RoofMatl_ClyTile',
                 'RoofMatl_Membran','RoofMatl_Metal','RoofMatl_Roll','Utilities_NoSeWa']
# Add missing columns to X_test with default value of 0
X_test = X_test.assign(**{col: 0 for col in missing_columns})

In [46]:
set(X_train.columns)-set(X_test.columns)

set()

In [47]:
print('X_train shape', X_train.shape)
print('X_test shape', X_test.shape)

X_train shape (1460, 287)
X_test shape (1459, 287)


In [48]:
X_test = X_test[X_train.columns]

# Result

In [49]:
X_train.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,TotalFlArea,2ndFlRatio,LivLotRatio,Home_QualCond,...,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SeasonSold_Autumn,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter
0,65.0,8450,7,5,2003,2003,196.0,4,3,4,3,0,706,0,150,856,1,856,854,0,1710,1,0,2,1,3,1,4,8,0,0,2003.0,2,548,3,3,0,61,0,0,0,0,0,0,2,2008,1710,0.997664,0.202367,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
1,80.0,9600,6,8,1976,1976,0.0,3,3,4,3,3,978,0,284,1262,1,1262,0,0,1262,0,1,2,0,3,1,3,6,1,3,1976.0,2,460,3,3,298,0,0,0,0,0,0,0,5,2007,1262,0.000000,0.131458,7.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
2,68.0,11250,7,5,2001,2002,162.0,4,3,4,3,1,486,0,434,920,1,920,866,0,1786,1,0,2,1,3,1,4,6,1,3,2001.0,2,608,3,3,0,42,0,0,0,0,0,0,9,2008,1786,0.941304,0.158756,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
3,60.0,9550,7,5,1915,1970,0.0,3,3,3,4,0,216,0,540,756,1,961,756,0,1717,1,0,1,0,3,1,4,7,1,4,1998.0,3,642,3,3,0,35,272,0,0,0,0,0,2,2006,1717,0.786681,0.179791,6.0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0
4,84.0,14260,8,5,2000,2000,350.0,4,3,4,3,2,655,0,490,1145,1,1145,1053,0,2198,1,0,2,1,4,1,4,9,1,3,2000.0,3,836,3,3,192,84,0,0,0,0,0,0,12,2008,2198,0.919651,0.154137,6.5,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [50]:
y_train.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [51]:
X_test.head()

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,CentralAir,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,TotalFlArea,2ndFlRatio,LivLotRatio,Home_QualCond,...,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_NA,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SeasonSold_Autumn,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter
0,80.0,11622,5,6,1961,1961,0.0,3,3,3,3,0,468.0,144.0,270.0,882.0,1,896,0,0,896,0.0,0.0,1,0,2,1,3,5,0,0,1961.0,1.0,730.0,3,3,140,0,0,0,120,0,0,0,6,2010,896,0.000000,0.077095,5.5,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
1,81.0,14267,6,6,1958,1958,108.0,3,3,3,3,0,923.0,0.0,406.0,1329.0,1,1329,0,0,1329,0.0,0.0,1,1,3,1,4,6,0,0,1958.0,1.0,312.0,3,3,393,36,0,0,0,0,0,12500,6,2010,1329,0.000000,0.093152,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
2,74.0,13830,5,5,1997,1998,0.0,3,3,4,3,0,791.0,0.0,137.0,928.0,1,928,701,0,1629,0.0,0.0,2,1,3,1,3,6,1,3,1997.0,2.0,482.0,3,3,212,34,0,0,0,0,0,0,3,2010,1629,0.755388,0.117787,5.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
3,78.0,9978,6,6,1998,1998,20.0,3,3,3,3,0,602.0,0.0,324.0,926.0,1,926,678,0,1604,0.0,0.0,2,1,3,1,4,7,1,4,1998.0,2.0,470.0,3,3,360,36,0,0,0,0,0,0,6,2010,1604,0.732181,0.160754,6.0,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
4,43.0,5005,8,5,1992,1992,0.0,4,3,4,3,0,263.0,0.0,1017.0,1280.0,1,1280,0,0,1280,0.0,0.0,2,0,2,1,4,5,0,0,1992.0,2.0,506.0,3,3,0,82,0,0,144,0,0,0,1,2010,1280,0.000000,0.255744,6.5,...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [52]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv',index=False)
y_train.to_csv('y_train.csv', index=False)